In [1]:
!pip install spacy --user
!python3 -m spacy download it_core_news_sm
!pip3 install -U spacy
!pip3 install spacy
!python -m spacy download it_core_news_sm

     |████████████████████████████████| 21.1 MB 60.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
     |████████████████████████████████| 21.1 MB 1.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [2]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
import pandas as pd

In [3]:
df= pd.read_csv('Books.csv')
df

,Unnamed: 0,_id,author,contents,publishing house,year
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df['Frasi']= df.apply(lambda row: nltk.sent_tokenize(row['contents']), axis=1)
df

,Unnamed: 0,_id,author,contents,publishing house,year,Frasi
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014,[Quel giorno era impossibile uscire a passeggi...
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904,"[In quel giorno era impossibile passeggiare., ..."


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import spacy
from spacy.lang.it.examples import sentences 
from tqdm.notebook import tqdm
tqdm.pandas()
nlp = spacy.load('it_core_news_sm')
nlp.max_length = 1064150 
pipe = nlp.pipe(df['contents'], n_process=2, batch_size=2000)

In [8]:
docs = [x for x in tqdm(pipe)]

0it [00:00, ?it/s]

In [9]:
df['spacy_doc'] = docs

In [10]:
df['POS_spacy'] = df['spacy_doc'].progress_apply(lambda x: [(y.text, y.pos_) for y in x])

  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
df['lemmatized'] = df['spacy_doc'].progress_apply(lambda x: " ".join([y.lemma_ for y in x if len(x)>1]))

  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
def filter_text_pos(x):
    final_pos_text = []
    for elem in x:
        for pos in pos_list:
            if elem[1] == pos:
                final_pos_text.append(elem[0])
    
    return " ".join(final_pos_text)

In [13]:
def clean(x):
  stop = stopwords.words("italian")
  stop = stop + list(string.punctuation)
  words  = [w for w in word_tokenize(x.lower()) if w.isalpha() and w not in stop]
  return " ".join(words)

In [14]:
df['clean']= df.apply(lambda row: nltk.word_tokenize(row['lemmatized']), axis=1)

In [15]:
df

,Unnamed: 0,_id,author,contents,publishing house,year,Frasi,spacy_doc,POS_spacy,lemmatized,clean
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014,[Quel giorno era impossibile uscire a passeggi...,"(Quel, giorno, era, impossibile, uscire, a, pa...","[(Quel, DET), (giorno, NOUN), (era, AUX), (imp...",quello giorno essere impossibile uscire a pass...,"[quello, giorno, essere, impossibile, uscire, ..."
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904,"[In quel giorno era impossibile passeggiare., ...","(In, quel, giorno, era, impossibile, passeggia...","[(In, ADP), (quel, DET), (giorno, NOUN), (era,...",in quello giorno essere impossibile passeggiar...,"[in, quello, giorno, essere, impossibile, pass..."


In [16]:
corpus_text = 'n'.join(df['lemmatized'])
data = []
# iterate through each sentence in the file
for i in sent_tokenize(corpus_text):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [17]:
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count=5,
                                 window=10,
                                 size=50,
                                 sample=0,
                                 batch_words=100) 
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 50, window = 5, sg = 1)

In [18]:
model1.init_sims(replace=True)
model1.wv.most_similar(positive=["giorno"])

[('momento', 0.8303847312927246),
 ('sera', 0.8223902583122253),
 ('testo', 0.8032968044281006),
 ('settimana', 0.7733487486839294),
 ('attimo', 0.766821563243866),
 ('gita', 0.7635208368301392),
 ('parola', 0.7524915933609009),
 ('altra', 0.7451623678207397),
 ('pasto', 0.7444232702255249),
 ('trionfare', 0.7383038997650146)]

In [19]:
model2.init_sims(replace=True)

In [20]:
model2.wv.most_similar(positive=["giorno"])

[('settimana', 0.9144492745399475),
 ('mese', 0.9023645520210266),
 ('mattina', 0.8934290409088135),
 ('sera', 0.8813530206680298),
 ('durante', 0.8796690702438354),
 ('seguente', 0.872551679611206),
 ('trascorrere', 0.8602589964866638),
 ('mattino', 0.8573805093765259),
 ('quindici', 0.8563436269760132),
 ('otto', 0.855502188205719)]

In [21]:
model2.wv.similarity('giorno','mattino')

0.8573805

In [22]:
similar = model2.wv.most_similar('mattino')
similar

[('villaggio', 0.9618738293647766),
 ('attesa', 0.9577943086624146),
 ('pomeriggio', 0.956247866153717),
 ('studio', 0.9503991007804871),
 ('ingresso', 0.9468292593955994),
 ('campana', 0.9464650750160217),
 ('pasto', 0.944447934627533),
 ('mattina', 0.944154679775238),
 ('ritornare', 0.9440949559211731),
 ('raggiungere', 0.9436430335044861)]

In [23]:
model2.init_sims(replace = True)
model2.save('word2vec_model_lemmi')
model = Word2Vec.load('word2vec_model_lemmi')
model.wv.most_similar('pomeriggio')

[('passeggiare', 0.9652459621429443),
 ('giro', 0.9624659419059753),
 ('classe', 0.9609475135803223),
 ('campana', 0.9607309103012085),
 ('ingresso', 0.9601903557777405),
 ('mattina', 0.957444429397583),
 ('mattino', 0.956247866153717),
 ('pasto', 0.9539223909378052),
 ('salotto', 0.9524473547935486),
 ('hay', 0.9500246644020081)]

In [24]:
model.wv.most_similar('mattino')

[('villaggio', 0.9618738293647766),
 ('attesa', 0.9577943682670593),
 ('pomeriggio', 0.9562479257583618),
 ('studio', 0.9503991007804871),
 ('ingresso', 0.9468292593955994),
 ('campana', 0.9464650750160217),
 ('pasto', 0.944447934627533),
 ('mattina', 0.944154679775238),
 ('ritornare', 0.9440949559211731),
 ('raggiungere', 0.9436430335044861)]

In [25]:
distance = model.wv.wmdistance(data[0],data[12])
distance

0.7182606411760721

In [26]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.05902161895875851

In [27]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.3849626655321539

In [28]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.09438961251074374

In [29]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.05321715244927705

DOC2VEC

In [30]:
import numpy as np
documents = list(df['lemmatized'][0])
documents2 = list(df['lemmatized'][1])

df = pd.DataFrame(documents,columns={'text'})
df2 = pd.DataFrame(documents2, columns={'text'})
df3 = df.append(df2)

In [31]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(df3.text)]
model_d2v = Doc2Vec(vector_size=5, dbow_words=1, dm=0, epochs=30,  window=2, seed=1337, min_count=1, alpha=0.001, min_alpha=0.025)
model_d2v.build_vocab(tagged_data)
model_d2v.save('doc2vec_model')

In [32]:
model_d2v = Doc2Vec(vector_size=5, window=2, min_count=1, workers=4)
model_d2v.build_vocab(tagged_data)

In [33]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(1029592, 0.9999510049819946),
 (342643, 0.999428927898407),
 (30412, 0.9991466999053955),
 (1941816, 0.9988551735877991),
 (150368, 0.9987491965293884),
 (762639, 0.9986454248428345),
 (350597, 0.9982286095619202),
 (1702624, 0.9982165098190308),
 (1741407, 0.9980721473693848),
 (1174673, 0.9980465769767761)]

In [34]:
tagged_data[787038]

TaggedDocument(words=['v'], tags=[787038])

In [35]:
tokens = 'Quel giorno era impossibile uscire a passeggio.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(837757, 0.9995853304862976),
 (499456, 0.9993590116500854),
 (546993, 0.9993494749069214),
 (1778417, 0.9989938139915466),
 (1345608, 0.998478889465332),
 (1440224, 0.9983773827552795),
 (189855, 0.9982995986938477),
 (1483236, 0.9981717467308044),
 (656510, 0.9980055093765259),
 (322780, 0.9977800250053406)]

In [36]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(1029592, 0.9999510049819946),
 (342643, 0.999428927898407),
 (30412, 0.9991466999053955),
 (1941816, 0.9988551735877991),
 (150368, 0.9987491965293884),
 (762639, 0.9986454248428345),
 (350597, 0.9982286095619202),
 (1702624, 0.9982165098190308),
 (1741407, 0.9980721473693848),
 (1174673, 0.9980465769767761)]

In [37]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.019499381050552805

In [38]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.09823028145734358

In [39]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.01919756497742915

In [40]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.015463513338922414

In [41]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
new_vector_a = model_d2v.infer_vector(sentence_a)
new_vector_b = model_d2v.infer_vector(sentence_b)

cosine_similarity = np.dot(new_vector_a, new_vector_b)/(np.linalg.norm(new_vector_a)* np.linalg.norm(new_vector_b))
cosine_similarity

-0.16876952

SENT2VEC

In [42]:
!pip install bert
!pip install pytorch_pretrained_bert
!pip install sent2vec
!pip install sentence_transformers

In [43]:
import tensorflow as tf
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import random
import numpy as np
import matplotlib.pyplot as plt
import math

In [44]:
import torch#pytorch
from transformers import AutoTokenizer, AutoModel#for embeddings
from sklearn.metrics.pairwise import cosine_similarity#for similarity

#download pretrained model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-italian-cased",output_hidden_states=True)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
#create embeddings
def get_embeddings(text,token_length):
  tokens=tokenizer(text,max_length=token_length,padding='max_length',truncation=True)
  output=model(torch.tensor(tokens.input_ids).unsqueeze(0),
               attention_mask=torch.tensor(tokens.attention_mask).unsqueeze(0)).hidden_states[-1]
  return torch.mean(output,axis=1).detach().numpy()

In [46]:
#calculate similarity
def calculate_similarity(text1,text2,token_length=20):
    out1=get_embeddings(text1,token_length=token_length)#create embeddings of text
    out2=get_embeddings(text2,token_length=token_length)#create embeddings of text
    sim= cosine_similarity(out1,out2)[0][0]
    print(sim)

In [47]:
calculate_similarity(documents[0],documents2[0])

0.9387808


In [48]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'

calculate_similarity(sentence_a,sentence_b)

0.9024403


In [49]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

calculate_similarity(sentence_a,sentence_b)

0.94164723


In [50]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

calculate_similarity(sentence_a,sentence_b)

0.80446357
